## Análise de Erros Termometro Araraquara
___
* 1) Análise de Erros

#### Regras:
* Status != Normal: considerado como erro automaticamente
* Status = Normal, mas com temperatura negativa ou acima de 65 graus: criar nova categoria Ex: "Erro Consistência"
#####	Obs: Em teoria, quando ocorre isso, já deveria vir com o status "ErroTO" e acionar o substituto, mas já vimos que nem sempre acontece

## Importar Bibliotecas 

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [77]:
df = pd.read_csv('C:/Users/Cristiano/OneDrive - Belago Technologies/Documentos/projeto_fnt/data/araraquara_zar_erros.csv')
df.head()


,regiao,operacao,original,temperatura,temperaturatermometro,amplitude,status,dataref,year,month,day
0,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,22.4,22.4,3.9,ErroTOCF,2024-03-06,2024,3,6
1,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,26.7,22.4,8.2,ErroTOCF,2024-03-06,2024,3,6
2,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,26.5,22.4,8.0,ErroTOCF,2024-03-06,2024,3,6
3,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,26.5,22.4,8.0,ErroTOCF,2024-03-06,2024,3,6
4,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,26.4,22.4,7.9,ErroTOCF,2024-03-06,2024,3,6


In [78]:
df.isnull().sum()

regiao                   0
operacao                 0
original                 0
temperatura              0
temperaturatermometro    0
amplitude                0
status                   0
dataref                  0
year                     0
month                    0
day                      0
dtype: int64

In [79]:
df_estatistica = df[['temperatura', 'temperaturatermometro', 'amplitude']]

In [80]:
df_estatistica.describe()

,temperatura,temperaturatermometro,amplitude
count,7174.000000,7174.000000,7174.000000
mean,28.360705,51.882172,11.215946
std,10.788757,42.742536,11.330341
min,-20.000000,-31.800000,0.000000
25%,21.600000,19.400000,0.700000
50%,26.000000,28.300000,7.400000
75%,35.500000,111.500000,20.400000
max,62.900000,111.500000,63.900000


In [100]:
# Calculando o percentual de erros por termômetro
df_summary = df.groupby(["original", "regiao", "operacao"]).size().reset_index(name="quantidade_erros")
df_summary["percentual_erros"] = (df_summary["quantidade_erros"] / df_summary["quantidade_erros"].sum()) * 100

In [102]:
fig = px.bar(
    df_summary,
    x="original",
    y="percentual_erros",
    color="regiao",
    hover_data=["operacao"],
    title="Percentual de Erros por Termômetro e Região",
    labels={"original": "Termômetro", "percentual_erros": "Percentual de Erros (%)"},
    text="percentual_erros",
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    yaxis_title="Percentual de Erros (%)",
    xaxis_title="Termômetro",
    legend_title="Região",
    barmode="group",
    width=900,  
    height=600
)

fig.show()

In [86]:
# Criando um boxplot para visualizar a dispersão
fig = px.box(
    df,
    x="original",
    y="temperatura",
    color="status",
    points="all", 
    title="Dispersão das Temperaturas por Termômetro",
    labels={"original": "Termômetro", "temperatura": "Temperatura (°C)"},
)

fig.update_layout(
    yaxis_title="Temperatura (°C)",
    xaxis_title="Termômetro",
    legend_title="Status",
)

# Exibindo o gráfico
fig.show()

In [93]:
# Calculando o percentual de erros por termômetro e tipo de erro
df_summary = (
    df.groupby(["original", "status", "regiao"])
    .size()
    .reset_index(name="quantidade_erros")
)
df_summary["percentual_erros"] = (
    df_summary["quantidade_erros"] / df_summary["quantidade_erros"].sum()
) * 100

fig = px.bar(
    df_summary,
    x="original",
    y="percentual_erros",
    color="status",
    template="plotly_dark",
    hover_data=["regiao", "quantidade_erros"],
    title="Percentual de Erros por Termômetro e Tipo de Erro",
    labels={"original": "Termômetro", "percentual_erros": "Percentual de Erros (%)"},
    text="percentual_erros",
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    yaxis_title="Percentual de Erros (%)",
    xaxis_title="Termômetro",
    legend_title="Tipo de Erro",
    barmode="group",
    width=900,  
    height=500,  
)

fig.show()


## Análise de Erros por Série Temporal

In [103]:
# Convertendo a coluna 'dataref' para o formato datetime
df["dataref"] = pd.to_datetime(df["dataref"])

In [104]:
# Agrupando os dados por data e status para calcular a quantidade de erros
df_time_series = (
    df.groupby(["dataref", "status"])
    .size()
    .reset_index(name="quantidade_erros")
)

In [106]:
# Criando o gráfico de série temporal
fig = px.line(
    df_time_series,
    x="dataref",
    y="quantidade_erros",
    color="status",
    title="Evolução dos Erros ao Longo do Tempo",
    labels={"dataref": "Data", "quantidade_erros": "Quantidade de Erros"},
    template="plotly_dark",
)

fig.update_layout(
    yaxis_title="Quantidade de Erros",
    xaxis_title="Data",
    legend_title="Tipo de Erro",
    width=1200,
    height=600,
)

# Exibindo o gráfico
fig.show()

## Resumo das Análises e Gráficos
#### Gráfico de Percentual de Erros por Termômetro

* Objetivo: Identificar quais termômetros têm maior incidência de erros em relação ao total.
* Gráfico: Barras agrupadas por termômetro, com percentuais de erros representados.
* Insights: Ajuda a priorizar a manutenção ou substituição dos termômetros mais problemáticos.
___
## Dispersão das Temperaturas por Termômetro (Boxplot)

* Objetivo: Analisar como os valores de temperatura estão dispersos e identificar outliers.
* Gráfico: Boxplot com pontos individuais, categorizados por status do erro.
* Insights: Facilita a visualização de variações e padrões anômalos em medições de temperatura.

___
## Percentual de Erros por Termômetro e Tipo de Erro

* Objetivo: Avaliar como diferentes tipos de erros contribuem para o total de problemas em cada termômetro.
* Gráfico: Barras agrupadas, coloridas por tipo de erro (status).
* Insights: Permite identificar os tipos de erro mais frequentes para cada termômetro, auxiliando no diagnóstico e resolução.

___
## Evolução Temporal dos Erros

* Objetivo: Monitorar como os erros variam ao longo do tempo, destacando tendências ou anomalias.
* Gráfico: Linhas temporais mostrando a quantidade de erros por tipo (status).
* Insights: Identifica picos de erros e possíveis relações com fatores externos, como sazonalidade ou eventos específicos.

___
## Principais Conclusões:
* **Termômetros Problemáticos:** Os gráficos de barras destacam quais termômetros ou operações apresentam maior percentual de erros.
* **Dispersão dos Dados:** O boxplot revela inconsistências nos dados de temperatura, apontando outliers e possíveis falhas nos sensores.
* **Distribuição de Erros por Tipo:** Analisando os erros por tipo, é possível focar em problemas específicos, como calibração ou manutenção.
* **Tendências Temporais:** A série temporal ajuda a prever períodos de maior ocorrência de erros e planejar ações preventivas.

___
## **Recomendações:**
1. **Manutenção Preventiva:** Priorizar os termômetros com maior percentual de erros ou maior amplitude de dispersão.
2. **Análise dos Tipos de Erro:** Investigar os erros mais frequentes para direcionar correções específicas.
3. **Monitoramento Contínuo:** Utilizar gráficos temporais para acompanhamento em tempo real e ajustes proativos.